In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from researchlib.single_import import *

In [3]:
train_x = np.random.random((1000, 30, 300)) # (batches, features, time)
train_y = np.random.random((1000, 300)) # (batches, features(1 dimension could be squeezed), time)

test_x = np.random.random((100, 30, 300))
test_y = np.random.random((100, 300))

In [4]:
train_dataset = FromNumpy(train_x, train_y, batch_size=128)
test_dataset = FromNumpy(test_x, test_y, batch_size=128)

In [5]:
model = builder([
    PositionEncoding(128),
    TimeDistributed(nn.Linear(30, 128)),
    MultiHeadAttention(128, 128),
    MultiHeadAttention(128, 128),
    MultiHeadAttention(128, 128),
    MultiHeadAttention(128, 128),
    TimeDistributed(nn.Linear(128, 1)),
])

In [6]:
runner = Runner(model, train_dataset, test_dataset, 'adam', 'mse')
#runner = Runner(model, train_dataset, test_dataset, 'adam', {'mse':5, 'mae':10})  # loss ensemble!

In [7]:
runner.fit(10, 1e-2)

  Epochs    train_loss     val_loss   
    1*        0.3442        0.3221    


    2*        0.1903        0.1018    


    3*        0.0973        0.0939    


    4*        0.0923        0.0894    


    5*        0.0873        0.0875    


    6*        0.0864        0.0855    


    7*        0.0844        0.0852    


    8*        0.0844        0.0850    


    9*        0.0839        0.0849    


   10*        0.0840        0.0846    

